# IMDB Dataset 50k Movie Review Text Classification practical









In [ ]:
# File and data handling
import os
import numpy as np
import pandas as pd

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Text preprocessing
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Sklearn utilities
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import confusion_matrix

# Deep learning - Keras
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, SpatialDropout1D, LSTM, Dense, Dropout, Activation
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data_path="/content/IMDB Dataset.csv"

In [ ]:
df=pd.read_csv(data_path)
df.head();

NameError: name 'pd' is not defined

In [ ]:
df.shape

(50000, 2)

In [ ]:
df=df.iloc[:10000]
df.head()
#integrated location iloc

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
df.shape

(10000, 2)

In [ ]:
df['review'][0]

"One of the other reviewers has mentioned that after watching just 1 Oz episode you'll be hooked. They are right, as this is exactly what happened with me.<br /><br />The first thing that struck me about Oz was its brutality and unflinching scenes of violence, which set in right from the word GO. Trust me, this is not a show for the faint hearted or timid. This show pulls no punches with regards to drugs, sex or violence. Its is hardcore, in the classic use of the word.<br /><br />It is called OZ as that is the nickname given to the Oswald Maximum Security State Penitentary. It focuses mainly on Emerald City, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. Em City is home to many..Aryans, Muslims, gangstas, Latinos, Christians, Italians, Irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.<br /><br />I would say the main appeal of the show is due to the fa

In [ ]:
df['sentiment'].value_counts()

,count
sentiment,
positive,5028
negative,4972


In [ ]:
df['sentiment'].value_counts().get('positive',0)


np.int64(5028)

df.isnull().sum()

In [ ]:
df.duplicated().sum()

np.int64(17)

In [ ]:
df.drop_duplicates(inplace=True)

df.duplicated().sum()

## Basic Preprocessing




1.   Remove tags - HTML
2.   Lower case
3.  remove stopwords



In [ ]:
import re
def remove_tags(raw_text):
    cleaned_text=re.sub(re.compile('<.*?>'),'',raw_text)
    return cleaned_text

In [ ]:
df['review']=df['review'].apply(remove_tags)

In [ ]:
df.head()

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. the filming tec...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive


In [ ]:
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
#Text normalization -- lowercase
df['review']= df['review'].apply(lambda x:x.lower())

In [ ]:
df['review'][0]

"one of the other reviewers has mentioned that after watching just 1 oz episode you'll be hooked. they are right, as this is exactly what happened with me.the first thing that struck me about oz was its brutality and unflinching scenes of violence, which set in right from the word go. trust me, this is not a show for the faint hearted or timid. this show pulls no punches with regards to drugs, sex or violence. its is hardcore, in the classic use of the word.it is called oz as that is the nickname given to the oswald maximum security state penitentary. it focuses mainly on emerald city, an experimental section of the prison where all the cells have glass fronts and face inwards, so privacy is not high on the agenda. em city is home to many..aryans, muslims, gangstas, latinos, christians, italians, irish and more....so scuffles, death stares, dodgy dealings and shady agreements are never far away.i would say the main appeal of the show is due to the fact that it goes where other shows wo

In [ ]:
#remove stopwords a an the but
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
sw_list=stopwords.words('english')
df['review']=df['review'].apply(lambda x: [item for  item in  x.split() if item not in sw_list]).apply(lambda x:" ".join(x))
# Remove stopwords: split each review into words, keep only those not in the stopword list, then rejoin

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df['review'][0]

"one reviewers mentioned watching 1 oz episode hooked. right, exactly happened me.the first thing struck oz brutality unflinching scenes violence, set right word go. trust me, show faint hearted timid. show pulls punches regards drugs, sex violence. hardcore, classic use word.it called oz nickname given oswald maximum security state penitentary. focuses mainly emerald city, experimental section prison cells glass fronts face inwards, privacy high agenda. em city home many..aryans, muslims, gangstas, latinos, christians, italians, irish more....so scuffles, death stares, dodgy dealings shady agreements never far away.i would say main appeal show due fact goes shows dare. forget pretty pictures painted mainstream audiences, forget charm, forget romance...oz mess around. first episode ever saw struck nasty surreal, say ready it, watched more, developed taste oz, got accustomed high levels graphic violence. violence, injustice (crooked guards who'll sold nickel, inmates who'll kill order g

In [ ]:
df.head()

,review,sentiment
0,one reviewers mentioned watching 1 oz episode ...,positive
1,wonderful little production. filming technique...,positive
2,thought wonderful way spend time hot summer we...,positive
3,basically there's family little boy (jake) thi...,negative
4,"petter mattei's ""love time money"" visually stu...",positive


In [ ]:
#2D structure suitable for deep learning model makes it robust
#no of rows , data of rows
X=df.iloc[:,0:1]
X

,review
0,one reviewers mentioned watching 1 oz episode ...
1,wonderful little production. filming technique...
2,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu..."
...,...
9995,"fun, entertaining movie wwii german spy (julie..."
9996,"give break. anyone say ""good hockey movie""? kn..."
9997,movie bad movie. watching endless series bad h...
9998,"movie probably made entertain middle school, e..."


In [ ]:

#1D structure for classification targets
y=df['sentiment']
y

,sentiment
0,positive
1,positive
2,positive
3,negative
4,positive
...,...
9995,positive
9996,negative
9997,negative
9998,negative


In [ ]:
#Converting positive into 1 and negative into 0
from sklearn.preprocessing import LabelEncoder
encoder=LabelEncoder()
y=encoder.fit_transform(y)

In [ ]:
y

array([1, 1, 1, ..., 0, 0, 1])

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=1)

In [ ]:
X_train.shape

(7986, 1)

In [ ]:
X_test.shape

(1997, 1)

In [ ]:
#applying Bag of words

from sklearn.feature_extraction.text import CountVectorizer
cv=CountVectorizer()

In [ ]:
X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()

In [ ]:
X_train_bow

# Apply Bag-of-Words vectorization:
# - Each review becomes a fixed-length vector the size of the total vocabulary
# - Each column represents a word, and each value is its frequency in that review
# - All reviews have the same vector length because they’re mapped to the same word space
# - Most values are zeros since any given review uses only a few words from the full vocab


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
model.fit(X_train_bow,y_train)

GaussianNB()

In [ ]:
y_pred=model.predict(X_test_bow)

In [ ]:
from sklearn.metrics import accuracy_score,confusion_matrix
accuracy_score(y_test,y_pred)

0.6324486730095142

In [ ]:
#Confusion matrix
confusion_matrix(y_test,y_pred)

array([[717, 235],
       [499, 546]])

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.59      0.75      0.66       952
           1       0.70      0.52      0.60      1045

    accuracy                           0.63      1997
   macro avg       0.64      0.64      0.63      1997
weighted avg       0.65      0.63      0.63      1997



*Your model is better at identifying
negative reviews than positive ones.

*It tends to underpredict positives — possibly because positive reviews are more diverse or subtle in language.






#RandomForest


In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8502754131196795

In [ ]:
cv=CountVectorizer(max_features=3000)
X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()
rf=RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8347521281922884

#N grams
*An n-gram is a sequence of n words from a text. It helps capture context and word combinations, not just isolated words.

In [ ]:
cv=CountVectorizer(ngram_range=(1,2),max_features=5000)
X_train_bow=cv.fit_transform(X_train['review']).toarray()
X_test_bow=cv.transform(X_test['review']).toarray()
rf=RandomForestClassifier()
rf.fit(X_train_bow,y_train)
y_pred=rf.predict(X_test_bow)
accuracy_score(y_test,y_pred)

0.8482724086129194


*cv = CountVectorizer(...)	Initializes the vectorizer with n-gram/bigram and vocab size settings

*cv.fit_transform(...)-------Learns the vocabulary from training reviews and transforms them into BoW vectors


*.toarray()----	Converts the sparse matrix to a dense NumPy array (for compatibility with models)

#TFIDF


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()


X_train_tfidf = tfidf.fit_transform(X_train['review']).toarray()
X_test_tfidf = tfidf.transform(X_test['review'])


rf = RandomForestClassifier()
rf.fit(X_train_tfidf,y_train)
y_pred=rf.predict(X_test_tfidf)
accuracy_score(y_test,y_pred)


0.8477716574862293

##Word2Vec


In [ ]:
!pip install gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 78.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.0
    Uninstalling scipy-1.16.0:
      Successfully uninstalled scipy-1.16.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [ ]:
import nltk
nltk.download('punkt')
from tensorflow.keras.preprocessing.text import Tokenizer
from gensim.models import Word2Vec

In [ ]:
df = pd.read_csv('/content/IMDB Dataset.csv')  # or your own preprocessed file
# Make sure reviews are tokenized (list of words), else tokenize:
df['tokens'] = df['review'].apply(lambda x: x.lower().split())  # simple split


In [ ]:
le = LabelEncoder()
df['label'] = le.fit_transform(df['sentiment'])  # 'positive'→1, 'negative'→0


# STEP 4: Train Word2Vec


In [ ]:
w2v_model = Word2Vec(sentences=df['tokens'], vector_size=100, window=5, min_count=2, workers=4)
w2v_model.train(df['tokens'], total_examples=len(df['tokens']), epochs=10)


(11002400, 12229610)

# STEP 5: Convert Reviews to Vectors

In [ ]:
def get_review_vector(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if len(vectors) > 0:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

df['review_vector'] = df['tokens'].apply(lambda x: get_review_vector(x, w2v_model))


In [ ]:
X = np.array(df['review_vector'].tolist())
y = df['label'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred = clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.8128
